In [1]:
import nltk
!pip3 install wikipedia-api
import wikipediaapi
from collections import defaultdict
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import numpy as np
nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import gutenberg
from nltk.corpus import stopwords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
allbooks=list(gutenberg.fileids())
corpus_data=[]
for i in range(len(allbooks)):
    corpus_data += list(gutenberg.words(allbooks[i]))
with open("/content/Analogy_dataset.txt", "r") as f:
    lines = f.read().splitlines()
analogies=[]
for i in range(len(lines)):
    analogies +=lines[i].split(" ")
analogies=analogies[:-1]

In [4]:
def get_k_sentences(word,k):
    wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI)

    p_wiki = wiki_wiki.page(word)
    txt=p_wiki.text
    sentences = txt.split(".")
    k_sentences = []
    for sent in sentences:
        check_word = sent.split(" ")
        if word in check_word and len(check_word) < 50:
            k_sentences.append(sent)
        if len(k_sentences) > k:
            break
    return k_sentences

In [5]:
def tokenised_additional_corpus(k):
    analogy_raw_corpus = []
    for analogy in analogies:
      if analogy != " ":
          analogy_raw_corpus += get_k_sentences(analogy,k)
    full_analogy_corpus = ' '.join([elem for elem in analogy_raw_corpus])
    data = re.sub(r'[,!?;-]+\n:', '.', full_analogy_corpus)
    data = nltk.word_tokenize(data)
    return data

In [6]:
def full_corpus(k):
    corpus=tokenised_additional_corpus(k)+corpus_data
    stop_words = set(stopwords.words('english'))
    pattern = re.compile("^[a-zA-Z]+$")
    data=[]
    for ch in corpus:
        ch = ch.lower()
        if pattern.match(ch) and ch not in stop_words and ch != '.':
                data.append(ch)
    return data

get full corpus data (after performing pre processing) = gutenburg corpus + k tokenized sentences

In [7]:
processed_corpus = full_corpus(6) #initailly assuming k=6 to check

In [9]:
freq_dist = nltk.FreqDist(word for word in processed_corpus)
print("Total words in corpus: ",len(processed_corpus) )
print("Size of vocabulary: ",len(freq_dist) )
print("Most frequent tokens: ",freq_dist.most_common(20) )
V=len(freq_dist)

Total words in corpus:  1040281
Size of vocabulary:  42488
Most frequent tokens:  [('shall', 11682), ('said', 9432), ('unto', 9010), ('lord', 8592), ('thou', 6759), ('one', 6291), ('man', 5615), ('thy', 5609), ('god', 5287), ('thee', 4807), ('ye', 4674), ('upon', 4589), ('would', 4054), ('come', 3644), ('could', 3595), ('like', 3470), ('came', 3341), ('day', 3330), ('king', 3158), ('little', 3065)]


In [10]:
def indices_info(processed_corpus):
    processed = sorted(list(set(processed_corpus)))
    index = 0
    wordxInd = {}
    Indxword = {}
    for word in processed:
        if word not in wordxInd: 
             wordxInd[word] = index
             Indxword[index] = word
             index += 1
    return wordxInd, Indxword

In [11]:
import itertools
wordxInd, Indxword = indices_info(processed_corpus)
print(dict(itertools.islice(wordxInd.items(), 5)))
print(dict(itertools.islice(Indxword.items(), 5)))

{'aaron': 0, 'aaronites': 1, 'ab': 2, 'aback': 3, 'abaddon': 4}
{0: 'aaron', 1: 'aaronites', 2: 'ab', 3: 'aback', 4: 'abaddon'}


In [12]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1

In [13]:
for x, y in get_windows(processed_corpus[2000:2010],3):
    print(f'{x}\t{y}')

['german', 'empire', 'weimar', 'nazi', 'germany', 'berlin']	republic
['empire', 'weimar', 'republic', 'germany', 'berlin', 'served']	nazi
['weimar', 'republic', 'nazi', 'berlin', 'served', 'scientific']	germany
['republic', 'nazi', 'germany', 'served', 'scientific', 'artistic']	berlin


In [14]:
def word_to_one_hot_vector(word, word2Ind, V):
    one_hot_vector = np.zeros(V)
    one_hot_vector[word2Ind[word]] = 1  
    return one_hot_vector

In [15]:
def context_words_to_vector(context_words, word2Ind, V):
    context_words_vectors = [word_to_one_hot_vector(w, word2Ind, V) for w in context_words]
    context_words_vectors = np.mean(context_words_vectors, axis=0) 
    return context_words_vectors

In [16]:
def get_training_example(words, C, word2Ind, V):
    for context_words, center_word in get_windows(words, C):
        yield context_words_to_vector(context_words, word2Ind, V), word_to_one_hot_vector(center_word, word2Ind, V)

In [17]:
def get_train_set():
   train_x =None
   train_y = None
   for context_words_vector, center_word_vector in get_training_example(processed_corpus, 2, wordxInd, V):
       context_words_vector =   np.reshape(context_words_vector, (V, 1))  
       center_word_vector   =   np.reshape(center_word_vector, (V, 1))
       if train_x is not None:
           train_x=np.append(train_x, context_words_vector, axis=1)
           train_y=np.append(train_y, center_word_vector, axis=1)
       else:
           train_x=context_words_vector
           train_y=center_word_vector
   return train_x,train_y

In [18]:
def initialize_model(N,V, random_seed=1):
    np.random.seed(random_seed)
    W1 = np.random.rand(N,V)
    W2 = np.random.rand(V,N)
    b1 = np.random.rand(N,1)
    b2 = np.random.rand(V,1)
    return W1, W2, b1, b2

In [19]:
def softmax(z):
    e_z = np.exp(z)
    sum_e_z = np.sum(e_z,axis=0,keepdims=True)
    return e_z/sum_e_z

In [20]:
def forward_prop(x, W1, W2, b1, b2):
    h = np.dot(W1,x) + b1
    h = np.maximum(0,h)
    z = np.dot(W2,h) + b2
    return z, h

In [21]:
def compute_cost(y, yhat):
    m=y.shape[1]
    logprobs = np.multiply(np.log(yhat),y)
    cost = - 1/m * np.sum(logprobs)
    cost = np.squeeze(cost)
    return cost

In [22]:
def back_prop(x, yhat, y, h, W1, W2, b1, b2):
    m=x.shape[1]
    l1 = np.dot(W2.T ,(yhat - y))
    l1 = np.maximum(0,l1)
    grad_W1 = np.dot(l1,x.T)/m
    grad_W2 = np.dot((yhat - y),h.T)/m
    grad_b1 = np.sum(l1,axis=1,keepdims=True)/m
    grad_b2 = np.sum((yhat - y),axis=1,keepdims=True)/m
    return grad_W1, grad_W2, grad_b1, grad_b2

In [24]:
#x, y = get_train_set()

In [25]:
def gradient_descent(N, num_iters,alpha, 
                     random_seed, initialize_model=initialize_model, 
                     forward_prop=forward_prop, 
                     softmax=softmax,
                     compute_cost=compute_cost, 
                     back_prop=back_prop):
  
    W1, W2, b1, b2 = initialize_model(N,V, random_seed=random_seed)
    iters = 0
    for i in range(num_iters+1):
        z, h = forward_prop(x, W1, W2, b1, b2)
        yhat = softmax(z)
        cost = compute_cost(y, yhat)
        if ( (iters+1) % 10 == 0):
            print(f"iters: {iters + 1} cost: {cost:.6f}")
        grad_W1, grad_W2, grad_b1, grad_b2 = back_prop(x, yhat, y, h, W1, W2, b1, b2)

        W1 = W1 - alpha * grad_W1
        W2 = W2 - alpha * grad_W2
        b1 = b1 - alpha * grad_b1
        b2 = b2 - alpha * grad_b2
        iters +=1 
        if iters == num_iters: 
            break
            
    return W1, W2, b1, b2